# Stanza pipeline

If `gatenlp` has been installed with the stanza extra (`pip install gatenlp[stanza]` or `pip install gatenlp[all]`) you can run a Stanford Stanza pipeline on a document and get the result as `gatenlp` annotations. 



In [1]:
from gatenlp import Document
from gatenlp.lib_stanza import AnnStanza
import stanza



In [3]:
# In order to use the English pipeline with stanza, the model has to get downloaded first
stanza.download('en')

INFO:stanza:Downloading default packages for language: en (English)...
INFO:stanza:File exists: /home/johann/stanza_resources/en/default.zip.
INFO:stanza:Finished downloading models and saved to /home/johann/stanza_resources.


In [7]:
doc = Document.load("https://gatenlp.github.io/python-gatenlp/testdocument2.txt")
doc

Document(This is just a sample document for experimenting with gatenlp. 
It mentions a few named entities like the persons Barack Obama, 
Albert Einstein and Wolfgang Amadeus Mozart and the geographical 
locations and countries America, United States of America, 
Hungary, the Atlantic Ocean, and Helskinki. 

It also contains mentions of various numbers and amounts like 
12 degrees, 12.83$, 25 km/h or fortytwo kilos and mentions
organizations and companies like the UNO, Microsoft, Apple, which
has an apple as it's logo, or Google. 


,features=Features({}),anns={})

## Annotating the document using Stanza

In order to annotate one or more documents using Stanza, first create a AnnStanza annotator object
and the run the document(s) through this annotator:

In [4]:
stanza_annotator = AnnStanza(lang="en")

INFO:stanza:Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| sentiment | sstplus   |
| ner       | ontonotes |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: sentiment
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [8]:
doc = stanza_annotator(doc)
doc

Document(This is just a sample document for experimenting with gatenlp. 
It mentions a few named entities like the persons Barack Obama, 
Albert Einstein and Wolfgang Amadeus Mozart and the geographical 
locations and countries America, United States of America, 
Hungary, the Atlantic Ocean, and Helskinki. 

It also contains mentions of various numbers and amounts like 
12 degrees, 12.83$, 25 km/h or fortytwo kilos and mentions
organizations and companies like the UNO, Microsoft, Apple, which
has an apple as it's logo, or Google. 


,features=Features({}),anns={'': AnnotationSet([Annotation(0,4,Token,features=Features({'id': 1, 'text': 'This', 'lemma': 'this', 'upos': 'PRON', 'xpos': 'DT', 'head': 5, 'deprel': 'nsubj', 'ner': 'O', 'Number': 'Sing', 'PronType': 'Dem'}),id=0), Annotation(0,62,Sentence,features=Features({}),id=11), Annotation(5,7,Token,features=Features({'id': 2, 'text': 'is', 'lemma': 'be', 'upos': 'AUX', 'xpos': 'VBZ', 'head': 5, 'deprel': 'cop', 'ner': 'O', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin'}),id=1), Annotation(8,12,Token,features=Features({'id': 3, 'text': 'just', 'lemma': 'just', 'upos': 'ADV', 'xpos': 'RB', 'head': 5, 'deprel': 'advmod', 'ner': 'O'}),id=2), Annotation(13,14,Token,features=Features({'id': 4, 'text': 'a', 'lemma': 'a', 'upos': 'DET', 'xpos': 'DT', 'head': 5, 'deprel': 'det', 'ner': 'O', 'Definite': 'Ind', 'PronType': 'Art'}),id=3), Annotation(15,21,Token,features=Features({'id': 5, 'text': 'sample', 'lemma': 'sample', 'upos': 'NOUN', 'xpos': 'NN', 'head': 5, 'deprel': 'compound', 'ner': 'O', 'Number': 'Sing'}),id=4), Annotation(22,30,Token,features=Features({'id': 6, 'text': 'document', 'lemma': 'document', 'upos': 'NOUN', 'xpos': 'NN', 'head': 11, 'deprel': 'root', 'ner': 'O', 'Number': 'Sing'}),id=5), Annotation(31,34,Token,features=Features({'id': 7, 'text': 'for', 'lemma': 'for', 'upos': 'SCONJ', 'xpos': 'IN', 'head': 7, 'deprel': 'mark', 'ner': 'O'}),id=6), Annotation(35,48,Token,features=Features({'id': 8, 'text': 'experimenting', 'lemma': 'experiment', 'upos': 'VERB', 'xpos': 'VBG', 'head': 5, 'deprel': 'acl', 'ner': 'O', 'VerbForm': 'Ger'}),id=7), Annotation(49,53,Token,features=Features({'id': 9, 'text': 'with', 'lemma': 'with', 'upos': 'ADP', 'xpos': 'IN', 'head': 9, 'deprel': 'case', 'ner': 'O'}),id=8), Annotation(54,61,Token,features=Features({'id': 10, 'text': 'gatenlp', 'lemma': 'gatenlp', 'upos': 'NOUN', 'xpos': 'NN', 'head': 7, 'deprel': 'obl', 'ner': 'O', 'Number': 'Sing'}),id=9), Annotation(61,62,Token,features=Features({'id': 11, 'text': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': '.', 'head': 5, 'deprel': 'punct', 'ner': 'O'}),id=10), Annotation(64,66,Token,features=Features({'id': 1, 'text': 'It', 'lemma': 'it', 'upos': 'PRON', 'xpos': 'PRP', 'head': 13, 'deprel': 'nsubj', 'ner': 'O', 'Case': 'Nom', 'Gender': 'Neut', 'Number': 'Sing', 'Person': '3', 'PronType': 'Prs'}),id=12), Annotation(64,298,Sentence,features=Features({}),id=52), Annotation(67,75,Token,features=Features({'id': 2, 'text': 'mentions', 'lemma': 'mention', 'upos': 'VERB', 'xpos': 'VBZ', 'head': 52, 'deprel': 'root', 'ner': 'O', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin'}),id=13), Annotation(76,77,Token,features=Features({'id': 3, 'text': 'a', 'lemma': 'a', 'upos': 'DET', 'xpos': 'DT', 'head': 17, 'deprel': 'det', 'ner': 'O', 'Definite': 'Ind', 'PronType': 'Art'}),id=14), Annotation(78,81,Token,features=Features({'id': 4, 'text': 'few', 'lemma': 'few', 'upos': 'ADJ', 'xpos': 'JJ', 'head': 17, 'deprel': 'amod', 'ner': 'O', 'Degree': 'Pos'}),id=15), Annotation(82,87,Token,features=Features({'id': 5, 'text': 'named', 'lemma': 'name', 'upos': 'VERB', 'xpos': 'VBN', 'head': 17, 'deprel': 'amod', 'ner': 'O', 'Tense': 'Past', 'VerbForm': 'Part'}),id=16), Annotation(88,96,Token,features=Features({'id': 6, 'text': 'entities', 'lemma': 'entity', 'upos': 'NOUN', 'xpos': 'NNS', 'head': 13, 'deprel': 'obj', 'ner': 'O', 'Number': 'Plu